<div class="alert alert-danger" role="alert">
    <span style="font-size:20px">&#9888;</span> <span style="font-size:16px">This is a read-only notebook! If you want to make and save changes, save a copy by clicking on <b>File</b> &#8594; <b>Save a copy</b>. If this is already a copy, you can delete this cell.</span>
</div>

In [ ]:
# Load in Python libraries
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

In [ ]:
# Temporary Developer's code
%load_ext autoreload
%autoreload 2

In [ ]:
# Add path of the folder 'resources' to the path from which we can import modules  
import sys
sys.path.append('../utilities')
from clustering.kmeans import execute_k_means, k_means_range


# Read in data

In [ ]:
# Load in the data
soccer_data = pd.read_csv("sample_inputs/soccer.csv")

In [ ]:
# Look at the top 10 rows
pd.set_option('display.max_columns', None) # Show all columns -- only use if needed and data is not extremely wide
soccer_data.head(10)

### Other type of optional pre-processing to be added 

# Perform K-means analysis for a specific number of clusters

In [ ]:
from clustering.kmeans import KMeansClustering
import seaborn as sns

sns.set_style("white")

variable_names = [
    'crossing', 'finishing', 'heading_accuracy', 'short_passing',
    'dribbling', 'free_kick_accuracy', 'sprint_speed', 'ball_control', 
    'reactions', 'agility', 'sliding_tackle'
]

result = KMeansClustering.execute_k_means(
    dataset=soccer_data, 
    variables=variable_names,
    num_clusters=3, 
    standardize_vars=True, 
    generate_charts=True,
    save_results_to_excel=True
)

# OK: resize the 3d scatterplot
# OK: display the seaborn plot
# OK: export results to excel (either having an option on the function or with a separated function "to_excel" see what is best)

# TODO: Include more clustering methods to this template.
# TODO: Include the dimensionality reduction methods to the notebooks (add as a requirement for the clustering template) 

# Perform K-means analysis for a range of clusters

In [ ]:
from clustering.kmeans import KMeansClustering

results = KMeansClustering.k_means_range(
    dataset=soccer_data, 
    variables=variable_names,
    min_clusters = 2, max_clusters = 5,
    standardize_vars=True, 
    generate_charts=True,
    save_results_to_excel=True
    
)